# Code 4-67 to Code 4-81

Get the ner data saved in 5_NER_news_entity_run

In [45]:
import pandas as pd
t1 = pd.read_csv("only_ner_tagged.csv")

We process the data into 2 lists one with space and one without space

In [46]:
####start from here
t2 = t1[t1.ner_words.str.len()>=4]
t2["ner_words1"] = t2["ner_words"].str.lower()
t2["ner_words2"] = t2["ner_words1"].str.replace(" ","")
ner_list_space = t2["ner_words1"].unique()
ner_list = pd.Series(ner_list_space).str.replace(" ","")
len(t1),len(t2),len(ner_list_space),len(ner_list)

C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


(22762, 21726, 12239, 12239)

Read the files generated in "6_Sparql_Wikidata"

In [47]:
wiki_names = pd.read_csv("wikidata\persons_india.csv")
wiki_places = pd.read_csv("wikidata\city_state_country.csv")
wiki_positions = pd.read_csv("wikidata\positions_india.csv")

In [48]:
wiki_names.head()

,item,itemLabel
0,http://www.wikidata.org/entity/Q55713,Anand Yadav
1,http://www.wikidata.org/entity/Q55716,Bhargavaram Viththal Varerkar
2,http://www.wikidata.org/entity/Q55719,Sarojini Vaidya
3,http://www.wikidata.org/entity/Q55735,Aroon Tikekar
4,http://www.wikidata.org/entity/Q55744,Vijay Tendulkar


In [49]:
##places in india and countries of word
wiki_places.head()

,city,cityLabel
0,http://www.wikidata.org/entity/Q1154,Jamnagar
1,http://www.wikidata.org/entity/Q9461,Khedbrahma
2,http://www.wikidata.org/entity/Q9894,Dharmapuri
3,http://www.wikidata.org/entity/Q11854,Rajkot
4,http://www.wikidata.org/entity/Q13221,Tezu


In [50]:
##positions of public importance in india
wiki_positions.head()

,postion,positionLabel
0,NaN,Chief Minister of Telangana
1,NaN,Chief Minister of Nagaland
2,NaN,Chief Minister of Odisha
3,NaN,Chief Minister of Punjab
4,NaN,Chief Minister of Bihar


Processing the columns and combining the files into a single dataframe. We then create non duplicate lists with and without spaces

In [51]:
wiki_names.columns = ["url","itemLabel"]
wiki_places.columns = ["url","itemLabel"]
wiki_positions.columns = ["url","itemLabel1"]
##removing words like "of Gujarat","of Delhi" to arrive at better similarity
wiki_positions["itemLabel"] = wiki_positions["itemLabel1"].str.replace('of [a-zA-Z\s]+','') 

wiki_names = wiki_names.drop_duplicates(["itemLabel"])
wiki_places = wiki_places.drop_duplicates(["itemLabel"])
wiki_positions = wiki_positions.drop_duplicates(["itemLabel"])


wiki_all = pd.concat([wiki_names,wiki_places,wiki_positions],axis=0)


C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [52]:
wiki_all["itemLabel1"] = wiki_all["itemLabel"].str.lower()
item_list_space = pd.Series(wiki_all["itemLabel1"].unique())
wiki_all["itemLabel2"] = wiki_all["itemLabel1"].str.replace(" ","")

item_list = pd.Series(item_list_space).str.replace(" ","")


We now vectorize the lists -  ner_list and item_list as well as ner_list_space,item_list_space . The lists with space get vectorized using word vectorizer (ner_list,item_list). The lists without space gets vectorized as character vectorizer (ner_list_space,item_list_space). We then concatenate the outputs into 2 single arrays - one for our shortlisted “named entities” and other for the wikidata corpus

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer = 'char',ngram_range = (4,6),min_df=0.0001)
ner_vect_char = vectorizer.fit_transform(ner_list)
wiki_vect_char = vectorizer.transform(item_list)

vectorizer1 = TfidfVectorizer(analyzer = 'word',ngram_range = (1,1),min_df=0.0001)
ner_vect_word = vectorizer1.fit_transform(ner_list_space)
wiki_vect_word = vectorizer1.transform(item_list_space)


for cosine similarity of ner_vect and wiki_vect using sparse matrix as these arrays are large

In [54]:
from scipy.sparse import  hstack

ner_vect = hstack([ner_vect_char,ner_vect_word])
wiki_vect = hstack([wiki_vect_char,wiki_vect_word]) 

In [55]:
wiki_vect_word.shape,wiki_vect_word.shape

((55548, 3532), (55548, 3532))

Get Cosine Similarity

In [56]:
from sklearn.metrics.pairwise import cosine_similarity
sim_vec = cosine_similarity(ner_vect,wiki_vect,dense_output=False)

In [57]:
sim_vec

<12239x55548 sparse matrix of type '<class 'numpy.float64'>'
	with 10967669 stored elements in Compressed Sparse Row format>

We consider it as a match if the cosine similarity is greater than 0.6. Since we have a sparse matrix, we convert that to a “coordinate” format (coo). “coo” formats store data in 3 arrays one for row, column and data

In [58]:
condition = (sim_vec >=.6)
coo = condition.tocoo()

df_matches = pd.DataFrame(columns = ["wiki_match_label","ner_match_label"])
wiki_match_label = [item_list[i] for i in coo.col]

ner_match_label = [ner_list[i] for i in coo.row]

df_matches["wiki_match_label"] = wiki_match_label
df_matches["ner_match_label"] = ner_match_label

df_matches.head()

,wiki_match_label,ner_match_label
0,malaikaarora,malaikaarorakhan
1,manishmanikpuri,drmanishmandal
2,kashmirsingh,kashmir
3,"rupsa,india",india
4,"bassi,india",india


In [59]:
len(df_matches)

49455

We match with the original dataset (t2) to and get the non-matched entities as our final filtered organization entities

In [60]:
t3 = pd.merge(t2,df_matches, left_on = "ner_words2", right_on = "ner_match_label", how = "left")
len(t3[t3.ner_match_label.isna()==True])

13149

Get counts of "ner_words1" which are not in negative lists

In [61]:
t3.loc[t3.ner_match_label.isna()==True,"ner_words1"].value_counts()

supreme court                                      222
congress                                           141
air india                                           94
world cup                                           86
open sans                                           77
aam aadmi party                                     56
india today                                         54
police                                              53
bharatiya janata party                              52
also                                                51
party                                               50
indian army                                         40
army                                                32
income tax                                          29
samajwadi party                                     29
us president donald trump                           29
high court                                          27
bombay high court                                   27
rajya sabh

In [62]:
##Output for further analysis

In [63]:
t3[t3.ner_match_label.isna()==True].to_csv("ner_filtered.csv")

In [64]:
chks = t3.loc[t3.ner_match_label.isna()==True,"ner_words1"].value_counts()
len(chks[chks>=10])

115

We can see a lot of organizations in this list. We need to filter the list for the final list of organizations

Three options are there

1. There are 115 unique words if we apply a filter of count>= 10. We could stop here and proceed with the rest of the analysis.
2. We could also inspect the entire list manually (All 8K+ records) and select the final list of organizations. 
3. We can also look at the fact that "orgnanizations" would contain less of meanigful English words than the regular "English" Phrases. An example is the word "US President" Vs "Aam Aadmi Party". The former has more "meaningful" content than the later in English. This is elaborated in the following step 

Below code is a script that  identifies the percentage of english words in every row here. 

Enchant ("https://pypi.org/project/pyenchant/) is a library that identifies english from non-english words. Demo given here

In [44]:
#!pip install pyenchant
#You can install pyenchant using "pip"

In [65]:
import enchant
word_checks = enchant.Dict("en_US")
print (word_checks.check("Hello"))
print (word_checks.check("Xyz"))

True
False


In [66]:
shrt_words = pd.DataFrame(t3.loc[t3.ner_match_label.isna()==True,"ner_words1"].value_counts()).reset_index()
shrt_words.columns = ["words","count"]

In [67]:
word_checks = enchant.Dict("en_US")
def chk_eng(word):
    tot_flag = 0
    for i in word.split():
        flag = word_checks.check(i)
        if(flag==True):
            tot_flag = tot_flag +1
    
    return tot_flag
    

In [68]:
shrt_words["eng_ind"] = shrt_words["words"].apply(chk_eng)
shrt_words["word_cnt"] = (shrt_words["words"].str.count(" "))+1
shrt_words["percent"] = shrt_words["eng_ind"]/shrt_words["word_cnt"]
shrt_words.to_clipboard()

The percent here is the "meaningful" english word percentage in each phrase. I used this as a filter while manually skimming throught the list

In [69]:
shrt_words

,words,count,eng_ind,word_cnt,percent
0,supreme court,222,2,2,1.000000
1,congress,141,1,1,1.000000
2,air india,94,1,2,0.500000
3,world cup,86,2,2,1.000000
4,open sans,77,2,2,1.000000
5,aam aadmi party,56,1,3,0.333333
6,india today,54,1,2,0.500000
7,police,53,1,1,1.000000
8,bharatiya janata party,52,1,3,0.333333
9,also,51,1,1,1.000000


I manually skimmed through this file of 8K records and arrived at the following words

- air india
- gandhi institute of medical
- red bull
- united airlines
- qatar airways
- patanjali research institute
- jet airways
- millennium school
- facebook
- microsoft
- cbfc
- maruti celerio
- facebook india md
- sony xperia xz premium
- foxx
- uber ceo travis kalanick
- bharti airtel
- airtel
- change.org
- uber india president amit jain
- emami
